# Calculating MCD DTW(Mel cepstral distortion - Dynamic time warping)

In this tutorial we are going to learn how to calculate MCD DTW between a synthesized audio and reference audio.

Mel cepstral distortion(MCD) is an objective measure of speech quality. MCD is calculated between a tts generated audio and a ground truth audio. Two audios are similar, if the MCD value between them is low. MCD of an audio with itself is 0.

MCD DTW is a modification of MCD that works with non aligned audio clips by using dynamic time warping cost matrix. The scale depends on factors such as mel extractor and reduction algorithm(mean of DTW cost or min DTW path cost). It is useful for comparing the model convergence trained on same training data.

In [ ]:
# Install necessary packages
!pip install librosa numpy matplotlib

In [ ]:
## Import libraries
import librosa
import librosa.display
import numpy as np
import math
import matplotlib.pyplot as plt
import os

### Define parameters for spectrogram generation.
In this section we define parameters required to generate our spectrograms. More information about these parameteres can found on librosa documentation for [stft](https://librosa.org/doc/main/generated/librosa.stft.html), [mels](https://librosa.org/doc/main/generated/librosa.filters.mel.html) and [mfcc](https://librosa.org/doc/main/generated/librosa.feature.mfcc.html) generation.

In [ ]:
## Mel params
n_fft=1024
hop_length=256
win_length=None
window='hann'
n_mels = 80

## Mfcc params
n_mfcc=n_mels

A little bit about these parameters:  
    - `n_fft`: Number of fft_components or length of windowed signal after padding in stft.  
    - `hop_length`: Number of audio samples between adjacent STFT columns.  
    - `window`: Window to use in stft.  
    - `win_length`: Length of the window to be used.  
    - `n_mels`: Number of number of Mel bands to generate.  
    - `n_mfcc`: Number of MFCCs to generate.

### Load and Visualize data
Lets apply the algorithm on a synthesized, ground truth audio pair for understanding.

Load audio files using [librosa](https://librosa.org/doc/main/generated/librosa.load.html)

In [ ]:
# Load files
gt_file = "audio_samples/MCD-DTW/gt/sample_0.wav"
synt_file = "audio_samples/MCD-DTW/fastpitch/sample_0.wav"

## Load wavs
gt_wav, gt_sr = librosa.load(gt_file)
synt_wav, synt_sr = librosa.load(synt_file)

Generate mel specs and MFCC(Mel frequency cepstral coefficient) for ground truth and generated audio

In [ ]:
## Generate spectrograms
gt_mels = librosa.feature.melspectrogram(gt_wav, sr=gt_sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, n_mels=n_mels)
synt_mels = librosa.feature.melspectrogram(synt_wav, sr=gt_sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, n_mels=n_mels)
## Convert Mel specs to log scale
gt_mels = librosa.power_to_db(gt_mels, ref=np.max)
synt_mels = librosa.power_to_db(synt_mels, ref=np.max)


## Generate MFCC
gt_mfcc = librosa.feature.mfcc(S=gt_mels, n_mfcc=n_mfcc)
synt_mfcc = librosa.feature.mfcc(S=synt_mels, n_mfcc=n_mfcc)
##Convert MFCC to log scale
gt_mfcc = librosa.power_to_db(gt_mfcc, ref=np.max)
synt_mfcc = librosa.power_to_db(synt_mfcc, ref=np.max)

## Visualize the sample audios

<audio controls src="audio_samples/MCD-DTW/gt/sample_0.wav" type="audio/ogg"></audio><br>
<p><strong>Ground truth audio<strong><p><br>
<audio controls src="audio_samples/MCD-DTW/fastpitch/sample_0.wav" type="audio/ogg"></audio><br>
<p><strong>Synthesized audio<strong><p>



<img src="imgs/riva-tts-MCD_DTW_mels.jpeg">

### Calculate DTW matrix

Dynamic time warping(DTW) measures similarity between two time series that are not in sync. DTW achieves it by finding the most optimal path to align two sequences of different length.

It uses dynamic programming to calculate the cost of every alignment path and chooses the path with least accumulated cost. The accumulated Cost matrix D at  $x_a$ and $y_b$ is the minimum distance between the points, where $x$ and $y$ are two time series. Formally cost matrix can be defined as:
  
$D(a,b) = min(D(a-1, b), D(a, b-1), D(a-1, b-1)) + c(x_{a}, y_{b})$  
$D(1, b) = \sum(c(1, y_{b}))$  
$D(a, 1) = \sum(c(x_{a}, 1))$ 
  
Where ***x*** and ***y*** are the audio signals and ***c*** is the log cost function we have defined.

We will use [DTW function from librosa](https://librosa.org/doc/main/generated/librosa.sequence.dtw.html) on MFCC. It returns DTW accumulated cost matrix and DTW optimum path.

Cost function for DTW.

In [ ]:
## Define the cost function for calculating DTW
def log_spec_dB_dist(x, y):
    log_spec_dB_const = 10.0 / math.log(10.0) * math.sqrt(2.0)
    diff = x - y
    return log_spec_dB_const * math.sqrt(np.inner(diff, diff))

In [ ]:
frames = synt_mels.shape[1]
mcd = 0
dtw_cost, dtw_min_path = librosa.sequence.dtw(gt_mfcc, synt_mfcc, metric=log_spec_dB_dist)

Reduction of DTW matrix can be done by either taking a mean of entire cost matrix or averaging DTW cost for the mininum cost path per frame. We will use the DTW cost along the min cost path.

In [ ]:
path_cost_matrix = dtw_cost[dtw_min_path[:, 0], dtw_min_path[:, 1]]
path_cost = np.sum(path_cost_matrix)
path_length = dtw_min_path.shape[0]
reduced_dtw_cost = path_cost/path_length

mcd = reduced_dtw_cost/frames

print(f"MCD_DTW is: {mcd}")

## MCD_DTW usecase

MCD is a very useful metric to compare the convergence of two models. Therefore in this section we will calcaulate the average MCD for files from two models. We will first put the above calculations in functions for better readability.

we now write a function to load audio files and generate mfcc for them

In [ ]:
def get_mfcc(filename):
    wav, sr = librosa.load(filename)
    
    mels = librosa.feature.melspectrogram(wav, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, n_mels=n_mels)
    mels_db = librosa.power_to_db(mels, ref=np.max)
    
    mfcc = librosa.feature.mfcc(S=mels_db, n_mfcc=n_mfcc)
    mfcc_db = librosa.power_to_db(mfcc, ref=np.max)
    
    return mfcc_db, mels_db

write functions for getting average cost of Dynamic time warping along a path.

In [ ]:
def extract_path_cost(D, wp):
    """
    Get the path cost from D(cost matrix), wp (warped path)
    :returns: sum of path cost 
    """
    path_cost = D[wp[:, 0], wp[:, 1]]
    return np.sum(path_cost)

def extract_frame_avg_path_cost(D, wp):
    path_cost = extract_path_cost(D, wp)
    path_length = wp.shape[0]
    frame_avg_path_cost = path_cost / float(path_length)
    return frame_avg_path_cost
 

write a function for calculating MCD for single file.

In [ ]:
def cal_mcd(gt_mfcc, synt_mfcc, cost_function, dtw_type='path_cost'):
    frames = synt_mfcc.shape[1]
    path_cost = 0
    
    # dynamic time warping for MCD
    dtw_cost, dtw_min_path = librosa.sequence.dtw(gt_mfcc, synt_mfcc, metric=cost_function)
    if dtw_type == 'mean':
        path_cost = np.mean(dtw_cost)
    else:
        path_cost = extract_frame_avg_path_cost(dtw_cost, dtw_min_path)
    
    mcd = path_cost / frames
    
    return mcd, frames

In [ ]:
def cal_mcd_dir(synt_dir, gt_dir):
    mcds = []
    
    synt_filenames = os.listdir(synt_dir)
    synt_filepaths = [os.path.join(synt_dir, filename) for filename in synt_filenames]
    synt_filepaths.sort()
    gt_filenames = os.listdir(gt_dir)
    gt_filepaths = [os.path.join(gt_dir, filename) for filename in gt_filenames]
    gt_filepaths.sort()

    for synt_audio, gt_audio in zip(synt_filepaths, gt_filepaths):
        synt_mfcc, synt_mel = get_mfcc(synt_audio)
        gt_mfcc, gt_mel = get_mfcc(gt_audio)
        
        mcd, _ = cal_mcd(gt_mfcc, synt_mfcc, log_spec_dB_dist)
        mcds.append(mcd)
    return mcds

### Calculate MCD DTW on synthesized files from each model

Get audio files for both models and compare them.

In [ ]:
%%capture --no-display
audio_dir_m1 = "audio_samples/MCD-DTW/fastpitch/"
audio_dir_m2 = "audio_samples/MCD-DTW/radtts/"
audio_dir_gt = "audio_samples/MCD-DTW/gt/"

mcds_m1 = cal_mcd_dir(audio_dir_m1, audio_dir_gt)
mcds_m2 = cal_mcd_dir(audio_dir_m2, audio_dir_gt)

In [ ]:
print(f"Average MCD for model 1 is: {sum(mcds_m1)/len(mcds_m1):.2f}")
print(f"Average MCD for model 2 is: {sum(mcds_m2)/len(mcds_m2):.2f}")

### Plotting MCD

Plot the MCD DTW values for both models, the model with lower MCD DTW value is closer to ground truth.

In [ ]:
x_axis = np.linspace(1, len(mcds_m1), 10) ## Define an x axis for for plotting in matplotlib
plt.plot(x_axis, mcds_m1, label="fastpitch")
plt.plot(x_axis, mcds_m2, label="radTTS")

plt.title("MCD DTW value for each file")
plt.ylabel("MCD_DTW value")

plt.grid()
plt.legend()
plt.show()

## Conclusion
<img src="imgs/riva-tts-MCD_DTW_final_comparision.jpeg">

From the graph above the value of MCD is greater for radtts audios than fastpitch audios, this is also reflected in the average MCD value for both models. Therefore we can conclude that fastpitch has better convergence than radtts. However we cannot evaluate the quality of audios generated by these models using MCD. MCD is a great tool for testing model convergence, but generated audios may have pronunciation and quality artefacts. Therefore MCD evaluation should be followed by a MOS(Mean opinion score) and CMOS(Comparative mean opinion scores) evaluation.